# Day-19: Model Interpretability tools

## Topics Covered:

- What is Model Interpretability
- How it works
- Benefits of Model Interpretability
- Types of Model Interpretability
- When and Where to Use These Tools
- Comparison & Visualization
- Example+ code snippet




## What is Model Interpretability?

- Model interpretability is the degree to which a human can understand the reasons behind a machine learning model's predictions. 
- Instead of simply getting a result, interpretability allows us to understand the "why" behind a model's decision. 
- It's about opening up the "black box" of complex models to gain insights into how they work.Especially when using black-box models like Random Forests, XGBoost, or Neural Networks, interpretation tools are critical.

- Intrinsic Interpretability: 
    - Some models, like Decision Trees or Linear Regression, are inherently transparent because their structure directly explains the prediction. 
    - For instance, in linear regression, the coefficient for each feature shows its impact on the target variable.

- Post-hoc Interpretability: 
    - This applies to models that are not easily interpretable, such as Random Forests or Neural Networks. 
    - Post-hoc methods are applied after the model has been trained to provide explanations for its decisions. 
    - The tools we've discussed (SHAP, LIME, Permutation Importance) are all post-hoc methods.

## How it Works?

Model interpretability tools analyze a trained model to reveal which features are most influential and how they affect the model's output. They do this by either looking at the model's internal structure (for simple models) or by probing the model's behavior with new data points (for complex models).

Intrinsic Interpretability: Some models, like Decision Trees or Linear Regression, are inherently transparent because their structure directly explains the prediction. For instance, in linear regression, the coefficient for each feature shows its impact on the target variable.

Post-hoc Interpretability: This applies to models that are not easily interpretable, such as Random Forests or Neural Networks. Post-hoc methods are applied after the model has been trained to provide explanations for its decisions. The tools we've discussed (SHAP, LIME, Permutation Importance) are all post-hoc methods.

## Benefits of Model Interpretability?

- Trust & Adoption: 
    - By explaining a model's behavior, we build trust with stakeholders, customers, and regulators. 
    - This is crucial for models used in high-stakes fields like finance and medicine.

- Debugging & Auditing: 
    - Interpretability helps us debug models by identifying hidden biases or errors.
    - If a model makes a bad prediction, we can use these tools to understand which features led to that error and fix the underlying issue in the data or model.

- Scientific Discovery: 
    - In research, a model's insights can lead to new scientific discoveries. 
    - For example, a model's feature importance could highlight a gene or a protein that was previously unknown to be related to a disease.

## Types of Model Interpretablity ?

Model interpretability can be categorized based on the scope of the explanation.

    - Global Interpretability: 
        - This provides an understanding of the model's overall behavior. It tells us which features are most important on average for the model's predictions across the entire dataset. Permutation Importance and SHAP's summary plots are examples of global interpretability.

    - Local Interpretability: 
        - This focuses on explaining a single, specific prediction. It helps us answer questions like, "Why did the model classify this specific person as high-risk?" LIME and SHAP's force plots are prime examples of local interpretability.

## Permutation Feature Importance

Permutation Importance is a model-agnostic and global method for calculating feature importance. It works by measuring the decrease in a model's performance when a single feature's values are randomly shuffled (permuted). If a feature is important, shuffling its values will significantly reduce the model's performance.

In [ ]:
from sklearn.inspection import permutation_importance

result = permutation_importance(model, X_test, y_test, n_repeats=10, random_state=42)
importances = result.importances_mean

# Visualize
plt.barh(X_test.columns, importances)
plt.title("Permutation Feature Importance")
plt.show()


- How it works:

1. Train a model on your data and record a baseline performance score on a validation set (e.g., accuracy, R-squared).

2. For a single feature, randomly shuffle its values in the validation set.

3. Re-run the model's predictions on the shuffled data and calculate the performance score again.

4. The difference between the baseline score and the new score indicates the importance of that feature. A larger drop means the feature is more important.

5. Repeat this process for all features.

## LIME (Local Interpretable Model-Agnostic Explanations)

LIME is a method that explains an individual prediction by approximating a complex model locally with a simple, interpretable one. It focuses on a single instance and creates a new, small dataset around it. This is particularly useful for explaining "black box" models, whereas we could easily understand the logic of an inherently interpretable model like a Decision Tree (Day 13) just by looking at its structure.

In [ ]:
from lime.lime_tabular import LimeTabularExplainer

explainer = LimeTabularExplainer(training_data=np.array(X_train),
                                 feature_names=X_train.columns,
                                 class_names=['No Churn', 'Churn'],
                                 mode='classification')

# Choose a row to explain
exp = explainer.explain_instance(X_test.iloc[1].values, model.predict_proba, num_features=5)
exp.show_in_notebook()


- How it works:

1. Select a single data point you want to explain.

2. Perturb this data point multiple times to create a new, local dataset.

3. Use the complex model to make predictions on this new dataset.

4. Train a simple, interpretable model (like linear regression or a decision tree) on this local dataset, with the complex model's predictions as the target.

5. The simple model's coefficients or rules then serve as the explanation for the single prediction of the complex model.



- LIME Advantages:

    - Model-Agnostic: 
        - Like Permutation Importance, it can be applied to any machine learning model.

    - Focus on Local Explanations: 
        - It's designed specifically to answer "Why did the model make this decision for this particular instance?"



## SHAP (SHapley Additive exPlanations)

SHAP is a unified framework for explaining any machine learning model's output. It's based on Shapley values from cooperative game theory, which fairly distributes the "payout" (the model's prediction) among the "players" (the features). SHAP provides both local and global explanations. It gives us a consistent and detailed way to understand feature importance for models, unlike the built-in feature importance we saw with Lasso regression on Day 18.

In [ ]:
import shap
import xgboost
from sklearn.model_selection import train_test_split

# Sample dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Train a model
model = xgboost.XGBClassifier().fit(X_train, y_train)

# SHAP values
explainer = shap.Explainer(model, X_train)
shap_values = explainer(X_test)

# Plot
shap.plots.beeswarm(shap_values)


- How it works (Simplified): 
1. For a single prediction, SHAP calculates the contribution of each feature to the final prediction by comparing it to the average prediction for the dataset.

- SHAP Advantages:

    - Consistent: 
        - SHAP values are mathematically guaranteed to be consistent, meaning if a feature's contribution increases, its SHAP value will also increase.

    - Local & Global: 
        - You can use SHAP to explain a single prediction (local) or to summarize feature importance across the entire dataset (global).

In [ ]:
! python -m venv env
! .\env\Scripts\activate
! python -m pip install --upgrade pip

# Install your packages
! pip install numba numpy shap scikit-learn pandas matplotlib seaborn

# Test it
! python
import shap
# Import necessary libraries
import pandas as pd
import shap
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_breast_cancer

# Load dataset
cancer = load_breast_cancer()
X = pd.DataFrame(cancer.data, columns=cancer.feature_names)
y = cancer.target

# Split data and train a model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Create a SHAP explainer for the trained model
explainer = shap.TreeExplainer(model)
# Get SHAP values for a single instance (the first one in our test set)
shap_values = explainer.shap_values(X_test.iloc[0, :])

# Visualize the local explanation
print("SHAP Local Explanation for a single test instance:")
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[1], X_test.iloc[0,:])

In [ ]:
! python.exe -m pip install --upgrade pip